In [0]:
#get the file name from the adf
fileName = dbutils.widgets.get('fileName')
#fileName='Product.csv'
fileNameWithoutExt = fileName.split('.')[0]
print(fileNameWithoutExt)


Product


In [0]:
import pyspark.sql.functions as F
#from pyspark.sql import *
#from datetime import datetme as dt

#Just change all the values here based on the resource name you have created in your environemnt and workspace.

sqlDbName = 'project2sharuDB'
dbUserName = 'Sharathkumar'
passwordKey = 'sqldbpasswordkey'
stgAccountSASTokenKey = 'sastokenforst'
landingFileName =fileName #'Product'  #dbutils.widgets.get('Product')
databricksScopeName ='project2sharuscope'
dbServer = 'sharath1234'
dbServerPortNumber ='1433'
storageContainer ='input1'
storageAccount='project2sharuazure'
landingMountPoint ='/mnt'


In [0]:

sas_token = "sv=2024-11-04&ss=bfqt&srt=sco&sp=rwdlacupyx&se=2025-07-10T14:42:57Z&st=2025-07-09T06:42:57Z&spr=https&sig=XSEmJRr24vfbZOIxJUzkiJMqsk%2F0sEir666MoWIy1Ok%3D"  
storage_account_name = storageAccount
container_name = storageContainer
mount_point = landingMountPoint

dbutils.fs.mount(
  source = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net/",
  mount_point = mount_point,
  extra_configs = {
    f"fs.azure.sas.{container_name}.{storage_account_name}.blob.core.windows.net": sas_token
  }
)


    




True

In [0]:
#connect to Azure SQL DB
# Connection details
jdbc_hostname = "sharath1234.database.windows.net"      # Replace with your server
jdbc_port = 1433
jdbc_database = sqlDbName                               # Replace with your database name
jdbc_username = dbUserName                  # SQL login with @yourserver
jdbc_password = "Sharkesh@#10"                  # Replace with your actual password

# JDBC URL
jdbc_url = f"jdbc:sqlserver://{jdbc_hostname}:{jdbc_port};database={jdbc_database};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"

# JDBC properties
connection_properties = {
    "user": jdbc_username,
    "password": jdbc_password,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

# Read from Azure SQL DB (replace dbo.TableName with your table)
df = spark.read.jdbc(url=jdbc_url, table="dbo.[FileDetailsFormat]", properties=connection_properties)

# Show the result
df.display()



FileNo,FileName,ColumnName,ColumnDateFormat,ColumnIsNull,ModifiedDate
1,Product,StartDate,MM-dd-yyyy,true,2012-06-18T22:34:09Z
1,Product,EndDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
1,Product,CreateDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
1,Product,ModifiedDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
2,ProductDescription,ModifiedDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
2,ProductDescription,StartDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
2,ProductDescription,EndDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
3,CustomerDetail,CreateDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z
3,CustomerDetail,ActiveDate,MM/dd/yyyy,true,2012-06-18T22:34:09Z


In [0]:
df1 = spark.read.csv('/mnt/landing/'+fileName, inferSchema=True, header=True)
#display(df1)

# Rule
errorFlag=False
errorMessage = ''
totalcount = df1.count()
print(totalcount)
distinctCount = df1.distinct().count()
print(distinctCount)
if distinctCount !=totalcount:
    errorFlag = True
    errorMessage = 'Duplication Found. Rule 1 Failed'
print(errorMessage)
    
# Rule 2
df2 = df.filter(df.FileName==fileNameWithoutExt).select('ColumnName','ColumnDateFormat' )
rows = df2.collect()
for r in rows:
    colName = r[0]
    colFormat =r[1]
    print(colName, colFormat)
    #display(df1.filter(F.to_date(colName, colFormat).isNull() ==True))
    formatCount =df1.filter(F.to_date(colName, colFormat).isNotNull() ==True).count()
    if formatCount != totalcount:
        errorFlag = True
        errorMessage = errorMessage +' DateFormate is incorrect for {} '.format(colName)
    else:
        print('All rows are good for ', colName)
print(errorMessage)

if errorFlag:
    dbutils.fs.mv('/mnt/landing/'+fileName,'/mnt/rejected/'+fileName )
    dbutils.notebook.exit('{"errorFlag": "true", "errorMessage":"'+errorMessage +'"}')
else:
    dbutils.fs.mv('/mnt/landing/'+fileName,'/mnt/staging/'+fileName )
    dbutils.notebook.exit('{"errorFlag": "false", "errorMessage":"No error"}')
